# Import everything we need

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import psycopg2 # Needed to get database errors when uploading dataframe
import sql
import sqlalchemy
# hides warning messages
import warnings
warnings.filterwarnings("ignore")

## Create dictionary for competitors & locations

In [10]:
search_term = { 
'Lufthansa' : ['Germany', 'Hungary', 'Bulgaria'],

'Air France' : ['Mexico' 'Netherlands', 'China', 'Vietnam', 'Thailand', 'Indonesia', 
                'Malaysia', 'Morocco', 'Tunisia', 'France'],

'Delta TechOps' : ['USA', 'Canada', 'Mexico', 'Argentina', 'Bolivia', 'Brazil', 'Chile', 
                    'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru', 'South Georgia', 
                    'Suriname', 'Uruguay', 'Venezuela'],
                   
'MTU Maintenance' : ['China', 'Serbia', 'Poland', 'Canada', 'USA', 'Malaysia', 'Germany']
}

# Looped request from API

In [11]:
import time
from functions_for_jobs import clean_location 
from functions_for_jobs import create_city_state_country 
from functions_for_jobs import assign_world_regions
from functions_for_jobs import create_id
from functions_for_jobs import translate_job_titles
from functions_for_jobs import clean_job_titles
from functions_for_jobs import clean_text
from functions_for_jobs import translate_job_description
from functions_for_jobs import delete_columns



url = "https://linkedin-jobs-search.p.rapidapi.com/"
headers = {
            "content-type": "application/json",
            "X-RapidAPI-Host": "linkedin-jobs-search.p.rapidapi.com",
            "X-RapidAPI-Key": "644312ac60msh39f1d2973e65c1ap1d2da5jsnbfb63b2251de"
            }
i = 1
for competitor in search_term:  

    for search_country in search_term[competitor]:
        
        while i < 15:
### Getting Data via API    
            payload = {
                        "search_terms": f'{competitor}',
                        "location": f'{search_country}', 									        
                        "page": f'{i}',     										        
                        "fetch_full_text": "yes"
                        }
           
            response = requests.request("POST", url, json=payload, headers=headers)
            jobs_list = json.loads(response.content)
            json.dumps(jobs_list, indent=3)
            job = pd.json_normalize(jobs_list, sep="_")
            print(job)
            print(competitor)
            print(search_country)
            time.sleep(15)   
            print('slept well and ready to go on')                                                                      # just 5 requests per minute are allowed
        

            if len(job) > 1:         
### Cleaning
                
                jobs = job[['company_name', 'job_title','job_location','posted_date','full_text']] # create DataFrame with necessary columns
                jobs['search_country'] = search_country              
                jobs = clean_location(jobs)
                jobs = create_city_state_country(jobs)
                jobs = assign_world_regions(jobs)
                jobs = create_id(jobs)
                jobs = translate_job_titles(jobs)
                jobs = clean_job_titles(jobs)
                jobs = clean_text(jobs)
                jobs['posted_date'] = pd.to_datetime(jobs['posted_date'], format="%Y-%m-%d")
                jobs = translate_job_description(jobs)
                jobs = delete_columns(jobs)
                
                
                
### Upload
                schema = os.getenv('schema')
                from sql import engine
                for row in range(len(jobs)):
                    try:
                        jobs.iloc[row:row+1, :].to_sql(name='jobs',                     # Name of DF table
                                                        con=engine,                     # Engine or connection
                                                        if_exists='append',             # Just add new values in existing table
                                                        schema=schema,                  # Use schema that was defined earlier in .sql
                                                        index=False,                    # Write DataFrame index as a column
                                                        chunksize=1,                    # Specify the number of rows in each batch to be written at a time
                                                        method='multi')                 # Pass multiple values in a single INSERT clause
                        print(f"row {row} of jobs was inserted successfully.")
                    except:
                        print(f"row {row} of jobs was ignored.")
                        continue
                i += 1 
                              
            else:
                i=1
                break  

                                              job_url  \
0   https://de.linkedin.com/jobs/view/internship-i...   
1   https://de.linkedin.com/jobs/view/internship-r...   
2   https://de.linkedin.com/jobs/view/praktikum-im...   
3   https://de.linkedin.com/jobs/view/praktikum-im...   
4   https://de.linkedin.com/jobs/view/verkehrsleit...   
5   https://de.linkedin.com/jobs/view/wahlstation-...   
6   https://de.linkedin.com/jobs/view/rechtsrefere...   
7   https://de.linkedin.com/jobs/view/praktikum-im...   
8   https://de.linkedin.com/jobs/view/praktikum-im...   
9   https://de.linkedin.com/jobs/view/praktikum-im...   
10  https://de.linkedin.com/jobs/view/praktikum-im...   
11  https://de.linkedin.com/jobs/view/praktikum-im...   
12  https://de.linkedin.com/jobs/view/certifying-s...   
13  https://de.linkedin.com/jobs/view/praktikum-im...   
14  https://de.linkedin.com/jobs/view/duales-studi...   
15  https://de.linkedin.com/jobs/view/referent-in-...   
16  https://de.linkedin.com/job

In [8]:
jobs.head()

,company_name,job_id,posted_date,job_description,job_title_en,world_region,city,state,country,search_country
0,Merkle EMEA,2022-06-13_Amsterdam_Affiliate/SEA Marketing C...,2022-06-13,,Affiliate/SEA Marketing Consultant,Europe,Amsterdam,North Holland,Netherlands,Netherlands
1,Merkle EMEA,2022-06-07_Rotterdam_Web Management & Delivery...,2022-06-07,,Web Management & Delivery - Adobe Team,Europe,Rotterdam,South Holland,Netherlands,Netherlands
2,Merkle EMEA,2022-06-06_Rotterdam_Manager Recruitment_Merkl...,2022-06-06,Company description: Merkle Merkle is the lead...,Manager Recruitment,Europe,Rotterdam,South Holland,Netherlands,Netherlands
3,Merkle EMEA,2022-06-10_Amsterdam_Technisch Web Analyst_Mer...,2022-06-10,Company description:About Merkle NederlandMerk...,Technical Web Analyst,Europe,Amsterdam,North Holland,Netherlands,Netherlands
4,Merkle EMEA,2022-06-13_Amsterdam_Medior/Senior Technical A...,2022-06-13,Company description:About Merkle NederlandMerk...,Medior/Senior Technical Analytics Consultant,Europe,Amsterdam,North Holland,Netherlands,Netherlands


In [114]:
jobs = clean_text(jobs)
jobs['posted_date'] = pd.to_datetime(jobs['posted_date'], format="%Y-%m-%d")
jobs = translate_job_description(jobs)
jobs = delete_columns(jobs)

In [15]:
jobs.head(25)

,company_name,job_id,posted_date,job_description,job_title_en,world_region,city,state,country,search_country
0,Air France,2022-05-25_Île-de-France_Manager d'équipe d'en...,2022-05-25,The aeronautical maintenance activity of the A...,Aircraft maintenance team manager,Europe,Île-de-France,NaN,France,France
1,Air France,2022-05-25_Roissy-en-France_Expert Engineering...,2022-05-25,The aeronautical maintenance activity of the A...,Expert Engineering Conception Cabine,Europe,Roissy-en-France,Île-de-France,France,France
2,Air France,2022-05-25_Orly_Manager de Chantier Moteur H/F...,2022-05-25,The aeronautical maintenance activity of the A...,Engine Site Manager,Europe,Orly,Île-de-France,France,France
3,Air France,2022-06-03_Roissy-en-France_Manager Contrôle N...,2022-06-03,The aeronautical maintenance activity of the A...,Non Destructive Testing Manager,Europe,Roissy-en-France,Île-de-France,France,France
4,Air France,2022-06-03_Roissy-en-France_Technicien Contrôl...,2022-06-03,The aeronautical maintenance activity of the A...,Aircraft CND Controller Technician,Europe,Roissy-en-France,Île-de-France,France,France


In [112]:
forbidden_job_titles = ['(m/f/d)', 'H/F', 'M/F', '(m/f)', 'F/M', '(d/f/m)', '(m/w/d)', '(m/f/diverse)', 'F/H', 
                '(F/M)', '( F/M)', 'm/f', '(m/f/x)', '(M/F/D)', '(all genders)', '(M/F)', '(d/m/f)', '(m / w / d)', 
                '(D/W/M)', '(f/m/d)', '(f/m/x)', '(f/m/diverse)', '(F / H)', '(He/She/They)', '(She/He/They)', 
                'F / H', '(H/F)', '(w/m/d)', '(3rd Shift)', '(1st Shift)','(On-Site', '( F/M)', '(Direct Hire)', 'REMOTE', 
                '(gn)', '[German speaking]', 'Remote', 'remote', '(Flex/Hybrid Options)', 'Freelance', '(flex/hybrid options)', 
                '(all genders)', '(m/w/d)-', '(REMOTE - US)', '(m / w / d)', 'Entry-Level', 'Entry-Level ', 'Sr. ', 'Sr.', ' H / F ',  'H / F', '()']
clean_job_title = pd.Series([])  

for row in range(len(jobs)):  
    clean_job_title[row] = jobs.loc[row, 'job_title_trans']

    for word in forbidden_job_titles:
        if word in jobs.loc[row, 'job_title_trans']:
            print(clean_job_title[row])
            print(word)
            clean_job_title[row] = clean_job_title[row].replace(word, '')

            
jobs.insert(5, "job_title_en", clean_job_title) 

Data Engineer / Developer BI BigData H / F
H / F
Data Engineer / Developer BI BigData H / F
H / F
Data Engineer / Developer BI BigData H / F
H / F
Expert SAP FICO H/F
H/F
Product Owner SAP Finance H/F
H/F
Cloud Engineer M/F
M/F
Cloud services engineer F/M
F/M
SAP Basis and Security Manager M/F
M/F
Functional Application Manager SAP Finance Accounting H/F
H/F
Perimeter Control Security Technical Architect (Border Control) M/F
M/F
RESPONSABLE COMMERCIAL (H/F)
H/F
RESPONSABLE COMMERCIAL ()
(H/F)
Director of Operations M/F
M/F
ALTERNANCE (ROISSY and ORLY) - HEAD OF QUALITY HEALTH SAFETY ENVIRONMENT M/F
M/F
Methods and Industrialization Engineer F/M
F/M
APPRENTICESHIP ENGINEER QSE CSR F/M
F/M
Industrial maintenance method engineer (M/F)
M/F
Industrial maintenance method engineer ()
(M/F)


In [116]:
schema = os.getenv('schema')
from sql import engine
for row in range(len(jobs)):
    try:
        jobs.iloc[row:row+1, :].to_sql(name='jobs',        # Name of DF table
                                        con=engine,                     # Engine or connection
                                        if_exists='append',             # Just add new values in existing table
                                        schema=schema,                  # Use schema that was defined earlier in .sql
                                        index=False,                    # Write DataFrame index as a column
                                        chunksize=1,                    # Specify the number of rows in each batch to be written at a time
                                        method='multi')                 # Pass multiple values in a single INSERT clause
        print(f"row {row} of jobs was inserted successfully.")
    except:
        print(f"row {row} of jobs was ignored.")

row 0 of jobs was inserted successfully.
row 1 of jobs was inserted successfully.
row 2 of jobs was inserted successfully.
row 3 of jobs was inserted successfully.
row 4 of jobs was inserted successfully.
row 5 of jobs was inserted successfully.
row 6 of jobs was inserted successfully.
row 7 of jobs was inserted successfully.
row 8 of jobs was inserted successfully.
row 9 of jobs was inserted successfully.
row 10 of jobs was inserted successfully.
row 11 of jobs was inserted successfully.
row 12 of jobs was inserted successfully.
row 13 of jobs was inserted successfully.
row 14 of jobs was inserted successfully.
row 15 of jobs was inserted successfully.
row 16 of jobs was inserted successfully.
row 17 of jobs was inserted successfully.
row 18 of jobs was inserted successfully.
row 19 of jobs was inserted successfully.
row 20 of jobs was inserted successfully.
row 21 of jobs was inserted successfully.
row 22 of jobs was inserted successfully.
row 23 of jobs was inserted successfully.


In [93]:
jobs.to_csv('Data/jobs/France_AirFrance.csv')

In [30]:
jobs = jobs.drop(columns='city', axis=1)
jobs = jobs.drop(columns='country', axis=1)
jobs = jobs.drop(columns='state', axis=1)

In [111]:
jobs = jobs.drop(columns='job_title_en', axis=1)

In [25]:
job_id = pd.Series([])

for row in range(len(jobs)):
    if  pd.api.types.is_float(jobs['city'][row]):
        print(jobs['city'][row])
        value = str(jobs.loc[row, 'posted_date'] + '_' + jobs.loc[row, 'country'] + '_' + jobs.loc[row, 'job_title'] + '_' + jobs.loc[row, 'company_name']) 
        job_id[row] = value
        print(value)
    else:
        print(row)
        print(type(jobs.loc[row, 'posted_date']))
        print(type(jobs.loc[row, 'city']))
        print(type(jobs.loc[row, 'job_title']))
        print(type(jobs.loc[row, 'company_name']))

        value = str(jobs.loc[row, 'posted_date'] + '_' + jobs.loc[row, 'city'] + '_' + jobs.loc[row, 'job_title'] + '_' + jobs.loc[row, 'company_name']) 
        job_id[row] = value
        print(value)

#jobs.insert(1, "job_id", job_id)

0
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
2022-06-03_Tunis_Customer Operations Analyst_LittleBig Connection
1
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
2022-05-23_Tunis_Senior Manager Customer Support_LittleBig Connection
2
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
2022-05-31_Tunis_Customer Success Specialist_ip-label group
nan
2022-05-23_Tunisia_Senior Java Saas Developer_LittleBig Connection


In [32]:
city = pd.Series([])
state = pd.Series([])
country = pd.Series([])

forbidden_words = {'Area','Greater','Metropolitan', 'region', 'Region', 'area', 
            'greater', 'metropolitan', 'Bay Area', 'bay area', 'Greater ', ' Area', ' area'}

us_list = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 
    'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 
    'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 
    'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 
    'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

for row in range(len(jobs)):
    print(len(jobs.loc[row,'location_list']))

    if len(jobs.loc[row,'location_list']) == 3:
        city[row] = jobs.loc[row, 'location_list'][0]
        state[row] = jobs.loc[row, 'location_list'][1]
        country[row] = jobs.loc[row, 'location_list'][2]
        print(f'{city} - 3')

    elif len(jobs.loc[row,'location_list']) == 2:  ## Washington, D.C. noch als Ausnahme hinzufügen
        if jobs.loc[row, 'location_list'][1] in us_list:
            city[row] = jobs.loc[row, 'location_list'][0]
            state[row] = jobs.loc[row, 'location_list'][1]
            country[row] = 'United States'
            print(f'{city} - 2 US')
        else:
            city[row] = jobs.loc[row, 'location_list'][0]       
            country[row] = jobs.loc[row, 'location_list'][1] 
            print(f'{city} - 2')  

    else:
        if jobs.loc[row,'location_list'] == jobs.loc[row, 'search_country']:
            country[row] = jobs.loc[row, 'location_list'][0]
        else:
            xx = jobs.loc[row,'location_list'][0].split(' ')
            if len(xx) > 1:
                print(len(xx))
            
                for word in xx:
                    
                    if word not in forbidden_words:
                        city[row] = ''.join(word)
                        print(f'{city[row]} if')
            else:
                city[row] = str(jobs.loc[row,'location_list'])
                print('else')
                
            country[row] = jobs.loc[row, 'search_country']


1
4
Paris if
3
0    Paris
1     Orly
dtype: object - 3
2
0            Paris
1             Orly
2    Île-de-France
dtype: object - 2
3
0               Paris
1                Orly
2       Île-de-France
3    Roissy-en-France
dtype: object - 3
1
4
Paris if
1
4
Paris if
3
0               Paris
1                Orly
2       Île-de-France
3    Roissy-en-France
4               Paris
5               Paris
6    Roissy-en-France
dtype: object - 3
3
0               Paris
1                Orly
2       Île-de-France
3    Roissy-en-France
4               Paris
5               Paris
6    Roissy-en-France
7                Orly
dtype: object - 3
3
0                Paris
1                 Orly
2        Île-de-France
3     Roissy-en-France
4                Paris
5                Paris
6     Roissy-en-France
7                 Orly
8    Villeneuve-le-Roi
dtype: object - 3
1
4
Paris if
1
4
Paris if
3
0                 Paris
1                  Orly
2         Île-de-France
3      Roissy-en-France
4            

# Single Request from API

search_term, location and page have to be assigned manually

In [ ]:
url = "https://linkedin-jobs-search.p.rapidapi.com/"
headers = {
            "content-type": "application/json",
            "X-RapidAPI-Host": "linkedin-jobs-search.p.rapidapi.com",
            "X-RapidAPI-Key": "a6d3c92a01msh4212f13f83cf821p182fedjsn1bd153fd656a"
            }
### Getting Data via API    
payload = {
            "search_terms": 'aeronautical engineer',
            "location": '', 									        
            "page": '1',     										        
            "fetch_full_text": "yes"
            }

response = requests.request("POST", url, json=payload, headers=headers)
jobs_list = json.loads(response.content)
json.dumps(jobs_list, indent=3)
job = pd.json_normalize(jobs_list, sep="_")
                                                                    # just 5 requests per minute are allowed

jobs_single = job[['company_name', 'job_title','job_location','posted_date','full_text']] # create DataFrame with necessary columns